In [1]:
import sys
sys.path.append(r'D:\repositories\baltika_hack')

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [11]:
import pandas as pd

# Загрузка данных
downtime_july_df = pd.read_excel(r'Простои за июль 2024.xlsx')
sensors_july_df = pd.read_excel(r'Датчики за июль 2024.xlsx')

# Приведение времени к минутам
downtime_july_df['start'] = pd.to_datetime(downtime_july_df['start']).dt.floor('min')
downtime_july_df['end'] = pd.to_datetime(downtime_july_df['end']).dt.floor('min')
sensors_july_df['start'] = pd.to_datetime(sensors_july_df['start']).dt.floor('min')
sensors_july_df['end'] = pd.to_datetime(sensors_july_df['end']).dt.floor('min')

# Создание временного ряда по часу
full_time_range = pd.date_range(start=downtime_july_df['start'].min(), end=downtime_july_df['end'].max(), freq='H')

# Развертывание данных по времени
downtime_expanded = pd.DataFrame({'timestamp': full_time_range})
sensors_expanded = pd.DataFrame({'timestamp': full_time_range})

# Добавляем duration, оставляя NaN

downtime_july_df['duration'] = downtime_july_df['duration'].fillna(0)
sensors_july_df['duration'] = sensors_july_df['duration'].fillna(0)

# Агрегация только числовых данных по часу
downtime_hourly = downtime_july_df.resample('H', on='start')[['duration']].sum().reset_index()
sensors_hourly = sensors_july_df.resample('H', on='start')[['duration']].sum().reset_index()

# Объединение данных
merged_df = pd.merge(downtime_hourly, sensors_hourly, on='start', suffixes=('_x', '_y'))

# Преобразование всех столбцов с типом datetime в числовой формат
for col in merged_df.select_dtypes(include=['datetime64[ns]']).columns:
    merged_df[col] = (merged_df[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

print(merged_df.head())


        start  duration_x  duration_y
0  1719831600   20.416666   26.516664
1  1719835200    0.000000   70.649998
2  1719838800    0.000000   94.449998
3  1719842400    0.000000   16.016665
4  1719846000   21.700000   73.399994


C:\Users\Максем\AppData\Local\Temp\ipykernel_29204\2861320725.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_time_range = pd.date_range(start=downtime_july_df['start'].min(), end=downtime_july_df['end'].max(), freq='H')
C:\Users\Максем\AppData\Local\Temp\ipykernel_29204\2861320725.py:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  downtime_hourly = downtime_july_df.resample('H', on='start')[['duration']].sum().reset_index()
C:\Users\Максем\AppData\Local\Temp\ipykernel_29204\2861320725.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sensors_hourly = sensors_july_df.resample('H', on='start')[['duration']].sum().reset_index()


In [12]:
merged_df.head(30)

,start,duration_x,duration_y
0,1719831600,20.416666,26.516664
1,1719835200,0.000000,70.649998
2,1719838800,0.000000,94.449998
3,1719842400,0.000000,16.016665
4,1719846000,21.700000,73.399994
5,1719849600,0.000000,50.649998
6,1719853200,29.433333,76.466664
7,1719856800,11.083333,39.133330
8,1719860400,43.433333,60.066663
9,1719864000,3.016666,61.983331
